<a href="https://colab.research.google.com/github/akshatsinha0407/PythonColab/blob/main/TextSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sun Nov 30 12:21:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install transformers[sentencepiece] dataset sacrebleu rouge_score py7zr -q

In [3]:
!pip install --upgrade acceleration
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

ERROR: Could not find a version that satisfies the requirement acceleration (from versions: none)
ERROR: No matching distribution found for acceleration
Found existing installation: transformers 4.57.3
Uninstalling transformers-4.57.3:
  Successfully uninstalled transformers-4.57.3
Found existing installation: accelerate 1.12.0
Uninstalling accelerate-1.12.0:
  Successfully uninstalled accelerate-1.12.0
  Using cached transformers-4.57.3-py3-none-any.whl.metadata (43 kB)
  Using cached accelerate-1.12.0-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.57.3-py3-none-any.whl (12.0 MB)
Using cached accelerate-1.12.0-py3-none-any.whl (380 kB)


In [4]:
!pip install rouge_score


In [5]:
# Install evaluate
!pip install evaluate

# Imports
import nltk
nltk.download('punkt')

from transformers import (
    pipeline,
    set_seed,
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq
)

from datasets import load_dataset  # still works
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch

import evaluate  # new way to load metrics

# Example: load Rouge metric
rouge = evaluate.load("rouge")
result = rouge.compute(predictions=["The cat is on the mat."], references=["The cat sat on the mat."])
print(result)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'rouge1': np.float64(0.8333333333333334), 'rouge2': np.float64(0.6), 'rougeL': np.float64(0.8333333333333334), 'rougeLsum': np.float64(0.8333333333333334)}


In [6]:
print(result)

{'rouge1': np.float64(0.8333333333333334), 'rouge2': np.float64(0.6), 'rougeL': np.float64(0.8333333333333334), 'rougeLsum': np.float64(0.8333333333333334)}


In [7]:
import evaluate

# Load Rouge metric
rouge = evaluate.load("rouge")

# Example usage
result = rouge.compute(
    predictions=["The cat is on the mat."],
    references=["The cat sat on the mat."]
)
print(result)


{'rouge1': np.float64(0.8333333333333334), 'rouge2': np.float64(0.6), 'rougeL': np.float64(0.8333333333333334), 'rougeLsum': np.float64(0.8333333333333334)}


In [8]:
from transformers import AutoModelForSeq2SeqLM , AutoTokenizer

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [9]:
model_ckpt = 'google/pegasus-cnn_dailymail'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# download & unzip data

!wget https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
!unzip summarizer-data.zip



--2025-11-30 12:24:32--  https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/summarizer-data.zip [following]
--2025-11-30 12:24:32--  https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/summarizer-data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7903594 (7.5M) [application/zip]
Saving to: ‘summarizer-data.zip.5’

summarizer-data.zip 100%[===================>]   7.54M  --.-KB/s    in 0.02s   

2025-11-30 12:24:33 (328 MB/s) - ‘summarizer-data.zip.5’ saved [7903

In [ ]:
from datasets import load_from_disk

dataset_samsum = load_from_disk('samsum_dataset')
print(dataset_samsum)


In [ ]:
# Check the lengths of each split
split_lengths = [len(dataset_samsum[split]) for split in dataset_samsum]
print(f"Split lengths: {split_lengths}")

# Print the feature/column names
print(f"Features: {dataset_samsum['train'].column_names}")

# Show an example dialogue and summary
print("\nDialogue:")
print(dataset_samsum['test'][1]['dialogue'])

print("\nSummary:")
print(dataset_samsum['test'][1]['summary'])


In [ ]:
def convert_examples_to_features(example_batch):
  input_encodings = tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)

  with tokenizer.as_target_tokenizer():
    target_encodings = tokenizer(example_batch['summary'], max_length=128 ,truncation=True)

  return {
      'input_ids': input_encodings['input_ids'],
      'attention_mask': input_encodings['attention_mask'],
      'labels': target_encodings['input_ids']
  }

In [ ]:
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched=True)

In [ ]:
dataset_samsum_pt['train']

In [ ]:
### Training
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
!pip install --upgrade wandb -q


In [ ]:
from transformers import TrainingArguments

trainer_args = TrainingArguments(
    output_dir='pegasus-samsum',
    num_train_epochs=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    warmup_steps=500,
    weight_decay=0.01,
    logging_steps=10,
    save_steps=int(1e6),
    gradient_accumulation_steps=16,
)


In [ ]:
trainer = Trainer(model= model_pegasus , args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt['train'],
                  eval_dataset=dataset_samsum_pt['validation']

                  )

In [ ]:
trainer.train()

In [ ]:
# Evaluation

def generate_batch_size_chunks(list_of_elements, batch_size):
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i:i+batch_size]

def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                                batch_size=16, device=device,
                                column_text='article',
                                column_summary='highlights'):
  article_batches = list(generate_batch_sized_chunks(dataset[column_text],batch_size))
  target_batches = list(generate_batch_sized_chunks(dataset[column_summary],batch_size))

  for article_batch, target_batch in tqdm(
      zip(article_batches, target_batches), total=len(article_batches)):

      inputs = tokenizer(article_batch , max_length=1024 , truncation=True,paddind="maz_Length",return_tensors="pt")
      summaries = model.generate(input_ids=inputs['input_ids'].to(device),
                                 attention_mask=inputs['attention_mask'].to(device),
                                 length_penalty=0.8, num_beams=8, max_length=128)


      decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                            clean_up_tokenization_spaces=True)
                           for s in summaries]
      meyric.add_batch(predictions=decoded_summaries, references=target_batch)

score = metrix.compute()
print(score)



In [ ]:
rough_names= ["rough1","rough2","roughL","rougeLsum"]
rouge_metix = load_metric('rouge')

In [ ]:
score = calculate_mertix_on_test_ds(
    datset_samsum['test'][0:10], rouge_metric, model_pegasus, tokenizer,batch_size=2, column_text='dialogue', column_summary='summary'
)

In [ ]:
rough_dict = dict((rn, score[rn].mid.fmeasure) for rn in rough_names)
pd.DataFrame(rough_dict, index=['pegasus'])

In [ ]:
model_pegasus.save_pretrained('pegasus-samsum-model')
tokenizer.save_pretrained('tokenizer')
tokenizer=AutoTokenizer.from_pretrained('/content/tokenizer')
#


In [ ]:
gen_kwargs = {"length_penalty":0.8,"num_beams":8,"max_length":128}

sample_text = datset_samsum["test"][0]["dialogue"]
reference = dataset_samsum["test"][0]["summary"]
pipe = pipeline("summarization",model="pegasus-samsum-model",tokenizer=tokenizer)

print("Dialogue:")
print(sample_text)

print("\nReference Summary:")
print(reference)


]qw3m,p['ml;
sdl;m
